<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250"/>

# TB - DevOps: Mise en place de cloud-native storage
## fio benchmark
**Note**: La version de fio utilisée pour chaque benchmark va être explicitement décrite tel que la license morale (section 4 [documentation](https://fio.readthedocs.io/en/latest/fio_doc.html#moral-license) fio) nous recommande.

## Local
Les résultats de cette section ont été généré à l'aide de fio version 3.16 à l'aide du script `run-all-jobs.sh`.

In [1]:
import json
# from: https://stackoverflow.com/a/3207973
from os import listdir
from os.path import isfile, join

fio_jobs_path = 'fio-jobs/local'
job_outputs_filename = [f for f in listdir(fio_jobs_path) if isfile(join(fio_jobs_path, f)) and 'output' in f]

In [2]:
job_outputs = []

# Load the results
for job_output in job_outputs_filename:
    f = open(f"{fio_jobs_path}/{job_output}")
    data = json.load(f)
    job_outputs.append(data)

print(job_outputs[0])

{'fio version': 'fio-3.16', 'timestamp': 1624716221, 'timestamp_ms': 1624716221209, 'time': 'Sat Jun 26 16:03:41 2021', 'global options': {'bs': '4K', 'iodepth': '256', 'direct': '1', 'ioengine': 'libaio', 'runtime': '120', 'numjobs': '4', 'size': '512M', 'name': 'raw-seqread', 'rw': 'read'}, 'jobs': [{'jobname': 'job1', 'groupid': 0, 'error': 0, 'eta': 0, 'elapsed': 121, 'job options': {'filename': '/dev/nvme0n1p7'}, 'read': {'io_bytes': 85691441152, 'io_kbytes': 83683048, 'bw_bytes': 714035839, 'bw': 697300, 'iops': 174325.156237, 'runtime': 120010, 'total_ios': 20920762, 'short_ios': 0, 'drop_ios': 0, 'slat_ns': {'min': 712, 'max': 318915, 'mean': 1345.356346, 'stddev': 891.144287}, 'clat_ns': {'min': 93938, 'max': 19756113, 'mean': 5872300.887986, 'stddev': 2146245.617705, 'percentile': {'1.000000': 1482752, '5.000000': 4014080, '10.000000': 4145152, '20.000000': 4227072, '30.000000': 4358144, '40.000000': 4489216, '50.000000': 5144576, '60.000000': 5406720, '70.000000': 7569408, '

## Cluster IICT
On souhaite déployer un container docker avec un PV dans le cluster IICT. Le container contient l'outil *fio* et un dossier *fio-jobs*.

Architecting-it a fait des [scripts](https://github.com/architectingit/k8sstorage/blob/main/perfraw.sh) pour réaliser ses tests. On va s'inspirer de leur script pour réaliser nos tests.
### Connection au cluster IICT
Connectez-vous au VPN de l'école, puis:
```bash
kubectl config get-contexts         # is ` iict ` listed ?
kubectl config use-context iict
```

### Manual
Si vous désirer allez plus rapidement, sautez à la section suivante (on applique une configuration dans le cluster et récupère les résultats):

```bash
kubectl apply -f iict-fio-benchmark-manual.yaml
```

Observons quel classe de stockage sont disponibles :
```bash
$ kubectl get storageclass
NAME                 PROVISIONER                                     RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
longhorn (default)   driver.longhorn.io                              Delete          Immediate           true                   170d
nfs-client           cluster.local/nfs-subdir-external-provisioner   Delete          Immediate           true                   114d
```

Nous allons donc utiliser un PVC de storageclass `longhorn`.

Ensuite, on déploie un container simple:

```bash
$ kubectl apply --namespace=mercado -f iict-fio-benchmark-manual.yaml
```

On vérifie que le container a bien démarré:
```bash
$ kubectl get pods --namespace=mercado
```

On ouvre une session interactive et on install fio:
```bash
$ kubectl exec --namespace=mercado -it $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}') -- /bin/sh
/ # apk add fio
/ # fio --version
$ exit # on sort de la session
```

On va copier un fio-job et un script permettant de run tous les tests:
```bash
# copy test script
$ kubectl --namespace=mercado cp docker/iict/run-all-jobs.sh $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/

# copy all jobs
$ kubectl --namespace=mercado cp docker/iict/fio-jobs/random-read.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
$ kubectl --namespace=mercado cp docker/iict/fio-jobs/random-readwrite.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
$ kubectl --namespace=mercado cp docker/iict/fio-jobs/random-write.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
$ kubectl --namespace=mercado cp docker/iict/fio-jobs/sequential-read.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
$ kubectl --namespace=mercado cp docker/iict/fio-jobs/sequential-readwrite.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
$ kubectl --namespace=mercado cp docker/iict/fio-jobs/sequential-write.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
```

Et on exécute les tests:
```bash
# ouvrir une session interactive
kubectl exec --namespace=mercado -it $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}') -- /bin/sh
sh run-all-jobs.sh
exit
```

On récupère tous les outputs:
```bash
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/random-read.fio-output fio-jobs/iict/random-read.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/random-readwrite.fio-output fio-jobs/iict/random-readwrite.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/random-write.fio-output fio-jobs/iict/random-write.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/sequential-read.fio-output fio-jobs/iict/sequential-read.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/sequential-readwrite.fio-output fio-jobs/iict/sequential-readwrite.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/sequential-write.fio-output fio-jobs/iict/sequential-write.fio-output
```

Une fois que l'on a terminé, on nettoie:
```bash
kubectl delete -f iict-fio-benchmark-manual.yaml
```

### Image

Nous allons déployer une image contenant les scripts de tests et un script d'exécution des tests en quelques lignes. On déploie :

```bash
$ kubectl apply -f iict-fio-benchmark.yaml
```

On peut observer l'exécution du container en consultant les logs (ou en allant sur l'UI rancher qui se rafraîchit automatiquement):
```bash
$ kubectl logs $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}') --namespace=mercado
```

Une fois que les logs affiche `"All jobs done"`, alors on peut récupérer les output et nettoyer les ressources:

```bash
$ sh iict-fio-benchmark-get-output.sh
$ kubectl delete -f iict-fio-benchmark.yaml
```

Maintenant on va visualiser nos données:


In [3]:
# from: https://stackoverflow.com/a/3207973
from os import listdir
from os.path import isfile, join

fio_jobs_path = 'docker/iict/fio-jobs'
job_outputs_filename = [f for f in listdir(fio_jobs_path) if isfile(join(fio_jobs_path, f))]

In [4]:
import json

job_outputs = []

# Load the results
for job_output in job_outputs_filename:
    f = open(f"fio-jobs/iict/{job_output}-output")
    data = json.load(f)
    job_outputs.append(data)

print(job_outputs[0])


{'fio version': 'fio-3.27', 'timestamp': 1624980492, 'timestamp_ms': 1624980492334, 'time': 'Tue Jun 29 15:28:12 2021', 'global options': {'name': 'read_bw', 'rw': 'randread', 'randrepeat': '0', 'verify': '0', 'ioengine': 'libaio', 'direct': '1', 'gtod_reduce': '1', 'bs': '128K', 'runtime': '30s', 'iodepth': '16', 'fdatasync': '0', 'size': '250G', 'ramp_time': '10s', 'filename': '/data/fiotest'}, 'jobs': [{'jobname': 'architecting-it-test3-read-bandwidth', 'groupid': 0, 'error': 0, 'eta': 0, 'elapsed': 42, 'read': {'io_bytes': 365559808, 'io_kbytes': 356992, 'bw_bytes': 11796057, 'bw': 11519, 'iops': 89.512746, 'runtime': 30990, 'total_ios': 2774, 'short_ios': 0, 'drop_ios': 0, 'slat_ns': {'min': 0, 'max': 0, 'mean': 0.0, 'stddev': 0.0, 'N': 0}, 'clat_ns': {'min': 0, 'max': 0, 'mean': 0.0, 'stddev': 0.0, 'N': 0}, 'lat_ns': {'min': 0, 'max': 0, 'mean': 0.0, 'stddev': 0.0, 'N': 0}, 'bw_min': 2048, 'bw_max': 25138, 'bw_agg': 100.0, 'bw_mean': 11650.262295, 'bw_dev': 4923.469383, 'bw_sampl

Note: [google](https://cloud.google.com/compute/docs/disks/benchmarking-pd-performance) persistent disk benchmark

#### Déploiement d'une nouvelle image

Pour mettre à jour l'image que le cluster peut utiliser pour benchmark (avec de nouveaux scénarios de tests ou des modifications), on tag un commit de ce repository avec, par exemple :

```bash
$ git tag # what tag can we use?
...
v0.1.4
v0.1.5
v0.1.6
v0.1.7

$ git tag v0.1.8
$ git push origin tag v0.1.8
```

On patiente un moment que l'image soit mise en ligne puis on peut réaliser notre benchmark.

**Note**: La github Action ne build que les images en `v0.*.*` pour le moment.
